In [5]:
#Assignment12.0 
# Method 1 Lab 12.0
# Part1
import cv2

cap = cv2.VideoCapture(0)
ret, frame = cap.read()
prevframe = frame
while True:
    ret, frame = cap.read()
    curframe = frame
    if ret:
        diff = cv2.absdiff(prevframe,curframe)
        cv2.imshow('Motion', diff)
        prevframe = curframe
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break
cv2.destroyAllWindows()
cap.release()

In [4]:
# Part2 another way to solve
import cv2

# cap = cv2.VideoCapture('run.avi')
cap = cv2.VideoCapture(0) #使用摄像头
ret, frame = cap.read()

prev2frame = frame
prevframe = frame

while True:
    ret, frame = cap.read()
    curframe = frame
    if ret:
        diff = cv2.absdiff(prev2frame,curframe)
        cv2.imshow('Motion', diff)
        prev2frame = prevframe
        prevframe = curframe
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break
cv2.destroyAllWindows()
cap.release()

In [89]:
# Method 2 Lab 12.2
# Part1
import numpy as np
import cv2
class avgDiff:
    def __init__(self,alpha,firstFrame):
        self.alpha  = alpha
        self.backGroundModel = firstFrame
    def getForeground(self,frame):
        self.backGroundModel =  frame * self.alpha + self.backGroundModel * (1 - self.alpha)
        return cv2.absdiff(self.backGroundModel.astype(np.uint8),frame)
def denoise(frame):
    frame = cv2.medianBlur(frame,5)
    frame = cv2.GaussianBlur(frame,(5,5),0)    
    return frame

In [90]:
# Part2
cam = cv2.VideoCapture(0)
ret,frame = cam.read()
if ret is True:
    backSub = avgDiff(0.001,denoise(frame)) # 0.001是效果最好的
    run = True
else:
    run = False

while(run):
    ret,frame = cam.read()

    if ret is True:
        cv2.imshow('Denoise Input',denoise(frame))
        foreGround = backSub.getForeground(denoise(frame))
        ret, mask = cv2.threshold(foreGround, 50, 255, cv2.THRESH_BINARY)
        cv2.imshow('mask',mask)

        key = cv2.waitKey(10) & 0xFF
    else:
        break

    if key == 27:
        break

cam.release()
cv2.destroyAllWindows()

In [102]:
# Method 3 Lab 12.3
#Part1
import cv2
import numpy as np
low = 4 #3
high = 4 #3
cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture('run.avi')
img_sum=0
diff_sum=0
count=0
ret, prev = cap.read()
prev=prev.astype(np.float32)
while (count<30):
    ret, frame = cap.read()
    if not ret:
        break
    frame=frame.astype(np.float32)
    img_sum+=frame
    diff_sum+=cv2.absdiff(prev,frame)
    prev = frame
    count=count+1
mean=img_sum/count
avgDiff=diff_sum/count
mean_show = mean.astype(np.uint8)
cv2.imshow('Background scene', mean_show)
cv2.waitKey(0)

27

In [103]:
#Part2
low_bound = mean - avgDiff*low
high_bound = mean + avgDiff*high

low_bound = low_bound.astype(np.uint8)
high_bound = high_bound.astype(np.uint8)
while (True):
    ret, frame = cap.read()
    if not ret:
        break
    
    if ret is True:
        frame_threshold = cv2.inRange(frame, low_bound, high_bound);
        ret, frame_threshold = cv2.threshold(frame_threshold, 100, 255, cv2.THRESH_BINARY_INV)
        cv2.imshow('Foreground image', frame_threshold)

        key = cv2.waitKey(10) & 0xFF
    else:
        break
        
    if key == 27:
        break

cv2.destroyAllWindows()

In [91]:
#Part3 background
import numpy as np
import cv2
#cap = cv2.VideoCapture('run.avi')
cap = cv2.VideoCapture(0)
fgbg = cv2.createBackgroundSubtractorMOG2()
while(1):
    ret, frame = cap.read()
    fgmask = fgbg.apply(frame)
    cv2.imshow('frame',fgmask)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()